In [ ]:
from lxml import html
import requests
import re

# Request the page
#page = requests.get('https://boards.4chan.org/vt/thread/72706478')
page = requests.get('https://boards.4chan.org/v/thread/673653330')
 
# Parsing the page
# (We need to use page.content rather than 
# page.text because html.fromstring implicitly
# expects bytes as input.)
tree = html.fromstring(page.content)  
 
# Get element using XPath
comments = tree.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "reply", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "postMessage", " " ))]')

In [ ]:
for comment in comments:
    # Get the text content of the element and strip leading and trailing whitespace
    text_content = comment.text_content().strip()
    # Print the result
    print(text_content)

In [ ]:
#pattern = re.compile(r'>>?\d+|>')
pattern = re.compile(r'>>?\d+')
text_contents = [re.sub(pattern, '', comment.text_content().strip()).replace('"', "'") for comment in comments]
text_contents

In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

In [ ]:
coments_sentiment = sentiment_pipeline(text_contents)

In [ ]:
coments_sentiment

In [ ]:
from collections import Counter
labels = [item['label'] for item in coments_sentiment]
label_counts = Counter(labels)

print(label_counts)

In [ ]:
import mysql.connector

connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="admin",
    database="sentiment_database"
)

In [ ]:
cursor = connection.cursor()

In [ ]:
query = f'DELETE FROM comments'

cursor.execute(query)

In [ ]:
for item in text_contents:
    query = f'INSERT INTO comments (text_content) VALUES ("{item}")'

    cursor.execute(query)

In [ ]:
query = "INSERT INTO comments (text_content) VALUES ('Something new or an actual sequel. No more worst of both worlds crap.')"
cursor.execute(query)

In [ ]:
query = "SELECT COUNT(*) FROM comments"
cursor.execute(query)

row_count = cursor.fetchone()[0]

print("Number of rows:", row_count)

In [ ]:
# Execute DELETE query
query = "DELETE FROM comments"
cursor.execute(query)

# Commit the transaction
connection.commit()

In [ ]:
query = "SELECT * FROM comments LIMIT 5"
cursor.execute(query)

# Fetch result
rows = cursor.fetchall()

# Display fetched rows
for row in rows:
    print(row)


In [ ]:
query = "SELECT text_content FROM comments LIMIT 5"
cursor.execute(query)

# Fetch result
rows = cursor.fetchall()

print(rows)

# Display fetched rows
for row in rows:
    print(row[0])


In [ ]:
rows

In [ ]:
connection.commit()
cursor.close()
connection.close()